# 웹크롤링을 해보자~
SYNOPSIS
    [.도움 (.help)]
    [.뉴스 (.news)]

DESCRIPTION
    .help: 도움말
    .news: 최신보안뉴스

EXTENSION
    [.실검 (.realtime)]
    [.날씨 지역 (.weather local)]
    [.환율 (.exchange)]
    [.운세 (.luck)]
    [.운세 띠 (.luck zodiac_name)]
    [.로또 (.lotto)]
    [.음식메뉴 (.foodmenu)]
    [.소라고동 내용 (.conch text)]
    [.노래추천 (.music)]


## 네이버 뉴스를 가지고 와보자~

In [22]:
var axios    = require('axios')
var cheerio  = require('cheerio')
var iconv    = require('iconv-lite');
var iconv    = new Iconv('CP949', 'utf-8//translit//ignore')
var log      = console.log

ReferenceError: Iconv is not defined

In [44]:
async function getNews() {
    var url = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url)
        data = data.data
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function() {
    str += `
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
<br>
`
    })
    console.log(str)
}
getNews()


https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=215&aid=0000879313&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
'��ϱ� ȯ��' �ᱹ �������Ρ�'1,900��' 3�� �߰� �ݿ�
<br>
[�ѱ�����TV ������ ����] ������ '�ڷγ�19' ���¿� ���� ���л����� ��ϱ� ȯ�� �䱸�� ����, 3�� �߰���������(�߰�)�ȿ� ���� ������ �ݿ��ϴ� ��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=015&aid=0004362083&date=&type=1&rankingSectionId=001&rankingSeq=2
<br>
"���� �����鼭"������� ���κ��� ���� '���ĸ���'
<br>
ģ�� ���� ����� ����� ��( ���� )�� ����� ���� "���� �����鼭"��� ���ε��� �����ϴ� ���� �߾��� �� ������ �ϰ� �ִ�. �� ���� 17�� ���� TB��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=023&aid=0003539716&date=&type=1&rankingSectionId=001&rankingSeq=3
<br>
[�ܵ�] ������� "���� ���� ������, ������ �Ǹ�������"
<br>
�����Ρ��ӵ��� �� ���ε�� û�ʹ� ���� "���ϰ��� ���� �ּ� ���ߴµ�... �׷��� �γ��ϸ� ��� ã�ڴ�" ������ ������� 17�� ������ ���Ͽܱ��Ⱥ���
<br>
<

In [45]:
async function getNews() {
    var url = 'https://news.naver.com/main/ranking/popularLike.nhn?subType=20'
    var NAVER = "https://news.naver.com"
    var str   = ""
    var data  = ""
    try {
        data = await axios.get(url)
        data = data.data
    } catch (err) {
        console.log(err)
    }
    const $ = cheerio.load(data, {decodeEntities: false});
    const $bodyList = $("div.likeitnews_item_text")
    $bodyList.each(function() {
    str += `
${NAVER + $(this).find('a.likeitnews_item_title').attr("href")}
<br>
${$(this).find('a.likeitnews_item_title').html()}
<br>
${$(this).find('div.likeitnews_item_lede').html()}
<br>
<br>
`
    })
    console.log(str)
}
getNews()


https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=215&aid=0000879313&date=&type=1&rankingSectionId=001&rankingSeq=1
<br>
'��ϱ� ȯ��' �ᱹ �������Ρ�'1,900��' 3�� �߰� �ݿ�
<br>
[�ѱ�����TV ������ ����] ������ '�ڷγ�19' ���¿� ���� ���л����� ��ϱ� ȯ�� �䱸�� ����, 3�� �߰���������(�߰�)�ȿ� ���� ������ �ݿ��ϴ� ��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=015&aid=0004362083&date=&type=1&rankingSectionId=001&rankingSeq=2
<br>
"���� �����鼭"������� ���κ��� ���� '���ĸ���'
<br>
ģ�� ���� ����� ����� ��( ���� )�� ����� ���� "���� �����鼭"��� ���ε��� �����ϴ� ���� �߾��� �� ������ �ϰ� �ִ�. �� ���� 17�� ���� TB��
<br>
<br>

https://news.naver.com/main/ranking/read.nhn?rankingType=popular_like&oid=023&aid=0003539716&date=&type=1&rankingSectionId=001&rankingSeq=3
<br>
[�ܵ�] ������� "���� ���� ������, ������ �Ǹ�������"
<br>
�����Ρ��ӵ��� �� ���ε�� û�ʹ� ���� "���ϰ��� ���� �ּ� ���ߴµ�... �׷��� �γ��ϸ� ��� ã�ڴ�" ������ ������� 17�� ������ ���Ͽܱ��Ⱥ���
<br>
<